In [1]:
import json
from collections import defaultdict
with open("data/train_500.txt", 'r') as f:
    lines = f.readlines()

entities = defaultdict(list)
samples = []
sample = []
texts = []
special_char = "<SPACE>"
text_entity_pair = []
entity_txt = ''
entity_name = ''
single_pair = defaultdict(list)
for i, line in enumerate(lines):
    line = line.strip()
    if len(line) == 0:
        if len(sample):
            samples.append(sample)
            text = "".join([t.split("\t")[0] for t in sample])
            text = text.replace(special_char, " ")
            texts.append(text)
            if entity_name:
                entities[entity_name].append("".join(entity_txt))
                single_pair[entity_name].append("".join(entity_txt))
            text_entity_pair.append(
                {"text": text, "entities": single_pair, "sample": sample})

        entity_txt = ''
        entity_name = ""
        single_pair = defaultdict(list)
        sample = []
        continue
    line = line.split(" ")

    if len(line) == 2:
        txt, tag = line
        sample.append("\t".join(line))
    elif len(line) == 1:
        tag = line[0]
        txt = " "
        sample.append("\t".join((special_char, line[0])))
    else:
        print(line)
    if tag == "O":
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
            single_pair[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_name = ""
    elif tag.startswith("B"):
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
            single_pair[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_txt += txt
        entity_name = tag.split("-")[-1]
    else:
        entity_txt += txt
        # entity_name = tag.split("-")[-1]

if len(sample):
    text = "".join([t.split("\t")[0] for t in sample])
    text = text.replace(special_char, " ")
    texts.append(text)
    samples.append(sample)
    if entity_name:
        entities[entity_name].append("".join(entity_txt))
        single_pair[entity_name].append("".join(entity_txt))
    text_entity_pair.append(
        {"text": text, "entities": single_pair, "sample": sample})



In [5]:
entities.keys()

dict_keys(['40', '4', '14', '5', '7', '11', '13', '8', '16', '29', '9', '12', '18', '1', '3', '22', '37', '39', '10', '36', '34', '31', '38', '54', '6', '30', '15', '2', '49', '21', '47', '23', '20', '50', '46', '41', '43', '48', '19', '52'])

In [4]:
from pypinyin import pinyin,Style

word = "中"
pinyin(word,style=Style.TONE3,heteronym=True)

[['。']]

In [4]:
from transformers.models.bert.tokenization_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/home/vocust001/pretrained_models/bert_rtb3")
text = "Bose SoundSport Free 真无线蓝牙耳机 运动耳机 博士防掉落耳塞 黑色"
# [t for t in text]

tokenizer.convert_tokens_to_ids(["a","b"])

[143, 144]

In [3]:
import torch.nn as nn
import torch

rnn = nn.GRU(10, 20, 2,batch_first=True)
input = torch.randn(30, 3, 10)
h0 = torch.randn(1, 3, 20)
output, hn = rnn(input)


In [35]:
hn.transpose(1,0).shape 

torch.Size([30, 2, 20])

In [15]:
embedding = nn.Embedding(7,32)
gru = nn.GRU(
            32,
            768,
            num_layers=1,
            batch_first=False,
            bidirectional=False,
        )
input = torch.LongTensor([[1,2,3,4],[2,3,1,5],[2,3,1,5]])
emb = embedding(input) 
# emb.shape 
output, hn = gru(emb)
for k,v in gru.state_dict().items():
    print(k,v.shape)

weight_ih_l0 torch.Size([2304, 32])
weight_hh_l0 torch.Size([2304, 768])
bias_ih_l0 torch.Size([2304])
bias_hh_l0 torch.Size([2304])


In [2]:
2304/768

3.0

In [28]:
from transformers import convert_tf_weight_name_to_pt_weight_name
convert_tf_weight_name_to_pt_weight_name("sk_emb/GRU/rnn/gru_cell/candidate/kernel")[0]

'GRU.rnn.gru_cell.candidate.weight'

In [6]:
from transformers import BertConfig,BertModel
model = BertModel.from_pretrained('/mnt/disk2/PythonProgram/NLPCode/PretrainModel/chinese_bert_base')
for k,v in model.state_dict().items():
    # if "sk_emb" in k and "adam_" not in k:
    print(k,v.shape)


Some weights of the model checkpoint at /mnt/disk2/PythonProgram/NLPCode/PretrainModel/chinese_bert_base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


embeddings.position_ids torch.Size([1, 512])
embeddings.word_embeddings.weight torch.Size([21128, 768])
embeddings.position_embeddings.weight torch.Size([512, 768])
embeddings.token_type_embeddings.weight torch.Size([2, 768])
embeddings.LayerNorm.weight torch.Size([768])
embeddings.LayerNorm.bias torch.Size([768])
encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias torch.Size([768])
encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias torch.Size([768])
encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias torch.Size([768])
encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
encoder.layer.0.intermediate.dense.weight 

In [40]:
import re
layer_number = re.compile("_(\d+)")
layer_number.sub(r".\1","encoder.layer_6.attention.self.key.weight")

'encoder.layer.6.attention.self.key.weight'

In [36]:
import torch
model = torch.load("../data/plome.pt")
for k,v in model.items():
    if "adam_v" in k or "adam_m" in k:
        continue
    # print(convert_tf_weight_name_to_pt_weight_name(k,start_prefix_to_remove="bert")[0],v.shape)
    if "bert" in k:
        print(convert_tf_weight_name_to_pt_weight_name(k)[0],v.shape)

bert.pooler.dense.weight torch.Size([768, 768])
bert.encoder.layer_9.output.dense.weight torch.Size([768, 3072])
bert.encoder.layer_9.intermediate.dense.weight torch.Size([3072, 768])
bert.encoder.layer_9.intermediate.dense.bias torch.Size([3072])
bert.encoder.layer_9.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer_8.output.dense.weight torch.Size([768, 3072])
bert.encoder.layer_8.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer_8.intermediate.dense.bias torch.Size([3072])
bert.encoder.layer_8.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer_9.attention.self.key.bias torch.Size([768])
bert.encoder.layer_8.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer_8.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer_7.output.dense.bias torch.Size([768])
bert.encoder.layer_7.intermediate.dense.weight torch.Size([3072, 768])
bert.encoder.layer_7.intermediate.dense.bias torch.Size([3072])
bert.encoder.lay

In [2]:
import torch
model1 = torch.load("./data/new_init.pt")
for k,v in model1.items():
    print(k,v.shape)

sk_emb.pyemb.weight torch.Size([7, 32])
sk_emb.GRU.weight_ih_l0 torch.Size([2304, 32])
sk_emb.GRU.weight_hh_l0 torch.Size([2304, 768])
sk_emb.GRU.bias_ih_l0 torch.Size([2304])
sk_emb.GRU.bias_hh_l0 torch.Size([2304])
py_emb.pyemb.weight torch.Size([30, 32])
py_emb.GRU.weight_ih_l0 torch.Size([2304, 32])
py_emb.GRU.weight_hh_l0 torch.Size([2304, 768])
py_emb.GRU.bias_ih_l0 torch.Size([2304])
py_emb.GRU.bias_hh_l0 torch.Size([2304])
encoder.layer.9.output.dense.weight torch.Size([768, 3072])
encoder.layer.9.intermediate.dense.weight torch.Size([3072, 768])
encoder.layer.9.intermediate.dense.bias torch.Size([3072])
encoder.layer.9.attention.output.dense.weight torch.Size([768, 768])
encoder.layer.8.output.dense.weight torch.Size([768, 3072])
encoder.layer.8.output.LayerNorm.bias torch.Size([768])
encoder.layer.8.intermediate.dense.bias torch.Size([3072])
encoder.layer.8.attention.self.value.weight torch.Size([768, 768])
encoder.layer.9.attention.self.key.bias torch.Size([768])
encoder.lay

In [7]:
from pypinyin import pinyin,lazy_pinyin,Style
lazy_pinyin("㐮",style=Style.TONE3)

['xiang1']

In [14]:
import torch
a = torch.tensor([[10,11,2,3,4,5,6],[10,11,2,3,4,5,6]])
mask = torch.tensor([[0,1,1,1,0,0,0],[0,1,1,1,1,1,0]]).type(torch.bool)
mask = ~mask.type(torch.bool)
torch.masked_fill(a,mask,torch.tensor(-100))
# mask

tensor([[-100,   11,    2,    3, -100, -100, -100],
        [-100,   11,    2,    3,    4,    5, -100]])

In [18]:
import re
sent_split = re.compile("[？？。！!]")
sent = "山本小铁子，日本女性BL漫画家与小说插画家，现居大阪。出生于1月4日，摩羯座，A型血。左撇子，喜欢鸟类，家中有两只鹦鹉，此外亦热衷于观赏棒球赛事及收集相关物品，经常将个人兴趣和生活经验融入原创的漫画作品之中。"
sents = sent_split.split(sent)
punct = sent_split.findall(sent)
punct = punct+[""]
new_sents = []
for s,p in zip(*(sents,punct)):
    new_sents.append(s+p)
# print(sents)
# print(punct)
new_sents

['山本小铁子，日本女性BL漫画家与小说插画家，现居大阪。',
 '出生于1月4日，摩羯座，A型血。',
 '左撇子，喜欢鸟类，家中有两只鹦鹉，此外亦热衷于观赏棒球赛事及收集相关物品，经常将个人兴趣和生活经验融入原创的漫画作品之中。',
 '']

In [3]:
from LAC import lac

In [4]:
import json
with open("data/entites.json",'r') as f:
    data = json.load(f)

for k,v in data.items():
    for k1,v1 in data.items():
        if k == k1:
            continue
        inter = set(v).intersection(set(v1))
        if len(inter)>0:
            print(k,k1,inter)


40 4 {'海报', '手账本', '冰激凌机', '水粉', '内存卡', '手表', '镜头', '摄像机', '充电器', '电视', '书桌', '投影仪', '文件夹', '支架', '电脑', '投影机', '显示器', '屏幕', '手台', '钢化膜', '白板', '笔记本', '清洁套装', '黑板', '硒鼓', '画纸', '云台', '塑料盒', '台式机', '话筒', '键盘', '鼠标', '全套配件', '分杯', '游戏机', '勾线笔', '水彩', '盒子', '网卡', '手机壳', '墨水', '数码相机', '设备', '水笔', '单反相机', '手机夹', '可擦笔', '打印机', '白板笔', '礼品', '外壳', '替换芯', '挂绳', 'U盘', '闪光灯', '铅笔', '冰淇淋机', '数据线', '冰箱', '笔芯', '橡皮', '水粉颜料', '相机', '耳机', '硅胶套', '收纳袋', '写字台', '主机', 'MP3', '电子称', '马克笔'}
40 5 {'手账'}
40 7 {'写字台', '书桌', '外壳'}
40 11 {'支架'}
40 13 {'指环', '大屏幕', '礼盒装'}
40 9 {'文件', '试卷', '自来水', '水粉画', '外壳'}
40 12 {'水粉', '不干胶'}
40 18 {'单反'}
40 22 {'挂绳', '内存卡', '指环', 'U盘', '墨水', '话筒', '支架', '笔芯', '屏幕', 'u盘', '软管'}
40 10 {'相机'}
40 31 {'挂绳'}
40 38 {'ipad'}
40 54 {'单反', '笔记本'}
4 40 {'海报', '手账本', '冰激凌机', '水粉', '内存卡', '手表', '镜头', '摄像机', '充电器', '电视', '书桌', '投影仪', '文件夹', '支架', '电脑', '投影机', '显示器', '屏幕', '手台', '钢化膜', '白板', '笔记本', '清洁套装', '黑板', '硒鼓', '画纸', '云台', '塑料盒', '台式机', '话筒', '键盘', '鼠标', '全套配件', '分杯', '游戏机', '勾线笔', '

In [4]:
from LAC.lac import LAC
lac = LAC()
a = lac.run(["智比奈特（ZBNET）网卡ZBE9602EF千兆双口多模SFP光纤网卡intel82576光纤网卡","智比奈特（ZBNET）网卡ZBE9602EF千兆双口多模SFP光纤网卡intel82576光纤网卡"])

In [5]:
for cut_text,cut_seg in a:
    print(cut_text,cut_seg)

['智比奈特', '（', 'ZBNET', '）', '网卡', 'ZBE9602EF', '千兆双口', '多模SFP', '光纤', '网卡', 'intel82576光纤', '网卡'] ['nz', 'w', 'n', 'w', 'n', 'n', 'nz', 'nz', 'n', 'n', 'n', 'n']
['智比奈特', '（', 'ZBNET', '）', '网卡', 'ZBE9602EF', '千兆双口', '多模SFP', '光纤', '网卡', 'intel82576光纤', '网卡'] ['nz', 'w', 'n', 'w', 'n', 'n', 'nz', 'nz', 'n', 'n', 'n', 'n']


In [2]:
import synonyms


In [17]:
choice_ent = synonyms.nearby("扩音器")
[word for word, pro in zip(*choice_ent) if 1 > pro > 0.65]

['麦克风', '对讲机', '话筒']

In [1]:
from LAC.lac import LAC